In [ ]:
import numpy as np
import time, os, sys
from urllib.parse import urlparse
import matplotlib.pyplot as plt
import matplotlib as mpl
import glob

from PIL import Image

# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'
from cellpose import utils, io, plot, models

In [ ]:
from skimage.measure import regionprops_table

In [ ]:
files = np.sort(glob.glob(
    "/u/home/f/f7xiesnm/project-zipursky/data/CTb_tracing_batch2/MAX_ctb*.tif"
))
files[:5], len(files)

In [ ]:
model = models.Cellpose(gpu=False, model_type='cyto')  # cyto or nuclei
channels = [[0,0]]
diameter = 15 # or None

for filename in files:
    print(filename)
    imgs = io.imread(filename) # read 
    # print(imgs.shape)
    
    nch, nx, ny = imgs.shape

    for i in range(nch):
        print(i)
        img = imgs[i]
        output_fname = filename.replace('.tif', f'_c{i}.tif') # as a basis
        output_fig   = filename.replace('.tif', f'_c{i}_viz.png')
        output_fcent = filename.replace('.tif', f'_c{i}_centroids.npy')
        
        # resize
        img_test = np.array(Image.fromarray(img).resize((512,512))) # resize

        # run cell pose
        mask, flow, style, diam = model.eval(img_test, diameter=diameter, channels=channels)

        # # save results as png
        # io.save_masks(img, mask, flow, output_fname)
        io.masks_flows_to_seg(img_test, mask, flow, diam, output_fname, channels)
        centroids = regionprops_table(mask, properties=['centroid', 'area'])
        np.save(output_fcent, centroids)

        # plot and see results
        fig = plt.figure(figsize=(20,10))
        plot.show_segmentation(fig, img_test, mask, flow[0], channels=channels)
        plt.tight_layout()
        plt.savefig(output_fig)
        plt.show()
        
    #     break
    # break